# Concatenating Tables with Set-Like Operations in `pyspark`

Now let's look at combining tables with `union`, `intersect`, and `except` in `pyspark`.

## Example - Auto Sales in Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Ops').config('spark.driver.host', 'localhost').getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

22/10/27 08:45:17 WARN Utils: Your hostname, kg3597wc201 resolves to a loopback address: 127.0.1.1; using 192.168.1.248 instead (on interface wifi0)
22/10/27 08:45:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/27 08:45:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from more_pyspark import to_pandas
sales_apr = spark.read.csv("./data/auto_sales_apr.csv", header=True, inferSchema=True)
sales_apr.collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9


In [3]:
sales_may = spark.read.csv("./data/auto_sales_may.csv", header=True, inferSchema=True)
sales_may.collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,20,14,6,24
2,2,Yolanda,19,10,28,17
3,3,Xerxes,11,27,17,9


## `UNION ALL` in `pyspark`

Both `union` and `unionAll` area actually `UNION ALL`

In [4]:
(sales_apr
 .union(sales_may)
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9
4,0,Ann,22,18,15,12
5,1,Bob,20,14,6,24
6,2,Yolanda,19,10,28,17
7,3,Xerxes,11,27,17,9


In [5]:
(sales_apr
 .unionAll(sales_may)
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12
1,1,Bob,19,12,17,20
2,2,Yolanda,19,8,32,15
3,3,Xerxes,12,23,18,9
4,0,Ann,22,18,15,12
5,1,Bob,20,14,6,24
6,2,Yolanda,19,10,28,17
7,3,Xerxes,11,27,17,9


## `UNION/UNION DISTINCT` in `pyspark`

Use `distinct` to get the usual `UNION/UNION DISTINCT`

In [6]:
(sales_apr
 .union(sales_may)
 .distinct()
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,2,Yolanda,19,8,32,15
1,3,Xerxes,12,23,18,9
2,1,Bob,19,12,17,20
3,0,Ann,22,18,15,12
4,1,Bob,20,14,6,24
5,2,Yolanda,19,10,28,17
6,3,Xerxes,11,27,17,9


## Adding a `month` column

As mentioned before, we really should add a month column here. Note that we need to use `lit` to add a *literal constant*

In [7]:
from pyspark.sql.functions import lit

(sales_apr.withColumn('month', lit('April'))
 .union(sales_may.withColumn('month', lit('May')))
 .collect()
) >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck,month
0,0,Ann,22,18,15,12,April
1,1,Bob,19,12,17,20,April
2,2,Yolanda,19,8,32,15,April
3,3,Xerxes,12,23,18,9,April
4,0,Ann,22,18,15,12,May
5,1,Bob,20,14,6,24,May
6,2,Yolanda,19,10,28,17,May
7,3,Xerxes,11,27,17,9,May


## Performing `INTERSECT`

Note that `intersect` and `intersectAll` are synonymous.

In [8]:
sales_apr.intersect(sales_may).collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,0,Ann,22,18,15,12


## Performing a set difference with `exceptAll`

In [9]:
sales_apr.exceptAll(sales_may).collect() >> to_pandas

,_c0,Salesperson,Compact,Sedan,SUV,Truck
0,1,Bob,19,12,17,20
1,2,Yolanda,19,8,32,15
2,3,Xerxes,12,23,18,9


## <font color="red"> Exercise 2 </font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Read in `pyspark` dataframes for April and May.
2. Check that the information about the date/month is present, and add a column if necessary.
3. `union` method to combine these 2 data frames into one combined `pyspark df`

In [10]:
!ls ./data | grep uber

uber-raw-data-apr14-sample.csv
uber-raw-data-aug14-sample.csv
uber-raw-data-jul14-sample.csv
uber-raw-data-jun14-sample.csv
uber-raw-data-may14-sample.csv
uber-raw-data-sep14-sample.csv


In [11]:
april = spark.read.csv("./data/uber-raw-data-apr14-sample.csv", header=True, inferSchema=True)
may = spark.read.csv("./data/uber-raw-data-may14-sample.csv", header=True, inferSchema=True)

In [12]:
april.take(5) >> to_pandas

,Date/Time,Lat,Lon,Base
0,4/18/2014 21:38:00,40.7359,-73.9852,B02682
1,4/23/2014 15:19:00,40.7642,-73.9543,B02598
2,4/10/2014 7:15:00,40.7138,-74.0103,B02598
3,4/11/2014 15:23:00,40.7847,-73.9698,B02682
4,4/7/2014 17:26:00,40.6460,-73.7767,B02598


In [13]:
april.union(may).collect() >> to_pandas

,Date/Time,Lat,Lon,Base
0,4/18/2014 21:38:00,40.7359,-73.9852,B02682
1,4/23/2014 15:19:00,40.7642,-73.9543,B02598
2,4/10/2014 7:15:00,40.7138,-74.0103,B02598
3,4/11/2014 15:23:00,40.7847,-73.9698,B02682
4,4/7/2014 17:26:00,40.6460,-73.7767,B02598
...,...,...,...,...
199995,5/23/2014 22:41:00,40.7341,-74.0084,B02682
199996,5/30/2014 6:53:00,40.6770,-74.0155,B02682
199997,5/3/2014 18:23:00,40.7710,-73.9636,B02682
199998,5/17/2014 13:16:00,40.7586,-73.9614,B02598


## <font color="red"> Exercise 3 -- Optional </font>

In the data folder, you will find 6 files that contain a sample 100,000 rows from the uber data for the month apr14-sep14.  Perform the following tasks:

1. Use `glob` to get all 6 file paths.
2. Use a regular expression to create a `lambda` function that pulls the month from the files.
3. Read the 6 `pyspark` dataframes into a `dict` with keys equal to the month name and values containing the corresponding data frame.
4. Use a dictionary comprehension to add a month column to each `df`.
5. Use the accumulator pattern and the `union` method to combine these 6 data frames into one combined `pyspark df`
6. Inspect the head and compute the number of rows (use the `count` method)

In [14]:
!ls ./data | grep uber

uber-raw-data-apr14-sample.csv
uber-raw-data-aug14-sample.csv
uber-raw-data-jul14-sample.csv
uber-raw-data-jun14-sample.csv
uber-raw-data-may14-sample.csv
uber-raw-data-sep14-sample.csv


In [15]:
# Your code here